## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541654/claude-3-opus-vs-google-gemini-vs-gpt-4-for-zero-shot-text-classification

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Importing and Installing Required Libraries 

In [13]:
!pip install anthropic
!pip install --upgrade google-cloud-aiplatform
!pip install openai

   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
    --------------------------------------- 0.1/4.2 MB 1.7 MB/s eta 0:00:03
   - -------------------------------------- 0.1/4.2 MB 1.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/4.2 MB 1.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/4.2 MB 1.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.4/4.2 MB 1.8 MB/s eta 0:00:03
   ---- ----------------------------------- 0.5/4.2 MB 1.6 MB/s eta 0:00:03
   ----- ---------------------------------- 0.6/4.2 MB 1.8 MB/s eta 0:00:03
   ------ --------------------------------- 0.7/4.2 MB 1.8 MB/s eta 0:00:02
   ------- -------------------------------- 0.8/4.2 MB 1.8 MB/s eta 0:00:02
   -------- ------------------------------- 0.9/4.2 MB 1.8 MB/s eta 0:00:02
   --------- ------------------------------ 1.0/4.2 MB 1.8 MB/s eta 0:00:02
   --------- ------------------------------ 1.0/4.2 MB 1.8 MB/s eta 0:00:02
   ---------- -----

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import anthropic
from openai import OpenAI
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

## Importing and Preprocessing the Dataset

In [ ]:
## Dataset download link
## https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?select=Tweets.csv

In [3]:
dataset = pd.read_csv(r"D:\Datasets\tweets.csv")
print(dataset.shape)
dataset.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
# Remove rows where 'airline_sentiment' or 'text' are NaN
dataset = dataset.dropna(subset=['airline_sentiment', 'text'])

# Remove rows where 'airline_sentiment' or 'text' are empty strings
dataset = dataset[(dataset['airline_sentiment'].str.strip() != '') & (dataset['text'].str.strip() != '')]

# Filter the DataFrame for each sentiment
neutral_df = dataset[dataset['airline_sentiment'] == 'neutral']
positive_df = dataset[dataset['airline_sentiment'] == 'positive']
negative_df = dataset[dataset['airline_sentiment'] == 'negative']

# Randomly sample records from each sentiment
neutral_sample = neutral_df.sample(n=34)
positive_sample = positive_df.sample(n=33)
negative_sample = negative_df.sample(n=33)

# Concatenate the samples into one DataFrame
dataset = pd.concat([neutral_sample, positive_sample, negative_sample])

# Reset index if needed
dataset.reset_index(drop=True, inplace=True)

# print value counts
print(dataset["airline_sentiment"].value_counts())

airline_sentiment
neutral     34
positive    33
negative    33
Name: count, dtype: int64


## Zero Shot Text Classification with Google Gemini Pro

In [44]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"D:\Datasets\grand-proton-411712-14fb02d06289.json"

In [45]:
model = GenerativeModel("gemini-pro")
config = {
    "max_output_tokens": 10,
    "temperature": 0.0,
}

def find_sentiment_gemini(tweet):

    content = """What is the sentiment expressed in the following tweet about an airline?
    Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
    tweet: {}""".format(tweet)

    responses = model.generate_content(
        content,
        generation_config= config,
    stream=True,
    )

    for response in responses:
        return response.text

In [46]:

%%time

all_sentiments = []

tweets_list = dataset["text"].tolist()

i = 0
exceptions = 0
while i < len(tweets_list):

    try:
        tweet = tweets_list[i]
        sentiment_value = find_sentiment_gemini(tweet)
        all_sentiments.append(sentiment_value)
        i = i + 1
        print(i, sentiment_value)

    except Except as e:
        print("===================")
        print("Exception occured", e)
        exception = exception + 1

print("Total exception count:", exceptions)


1 positive
2 neutral
3 negative
4 neutral
5 negative
6 positive
7 neutral
8 neutral
9 positive
10 negative
11 neutral
12 neutral
13 neutral
14 positive
15 neutral
16 negative
17 neutral
18 neutral
19 positive
20 negative
21 negative
22 negative
23 negative
24 neutral
25 positive
26 neutral
27 negative
28 neutral
29 neutral
30 neutral
31 neutral
32 neutral
33 neutral
34 neutral
35 positive
36 positive
37 positive
38 positive
39 positive
40 positive
41 positive
42 neutral
43 positive
44 positive
45 positive
46 positive
47 positive
48 positive
49 positive
50 positive
51 negative
52 positive
53 positive
54 positive
55 positive
56 positive
57 positive
58 positive
59 positive
60 positive
61 positive
62 negative
63 positive
64 positive
65 negative
66 negative
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 negative
74 negative
75 negative
76 negative
77 negative
78 negative
79 negative
80 negative
81 negative
82 negative
83 negative
84 negative
85 negative
86 negati

In [47]:
accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
print("Accuracy:", accuracy)

Accuracy: 0.78


## Zero Shot Text Classification with GPT-4

In [58]:
client = OpenAI(
    # This is the default and can be omitted
    api_key = os.environ.get('OPENAI_KEY2'),
)

In [62]:
def find_sentiment_gpt(tweet):
    
    content = """What is the sentiment expressed in the following tweet about an airline?
    Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
    tweet: {}""".format(tweet)

    sentiment = client.chat.completions.create(
      model= "gpt-4",
      temperature = 0,
      max_tokens = 10,
      messages=[
            {"role": "user", "content": content}
        ]
    )

    return sentiment.choices[0].message.content

In [63]:

%%time

all_sentiments = []

tweets_list = dataset["text"].tolist()

i = 0
exceptions = 0
while i < len(tweets_list):

    try:
        tweet = tweets_list[i]
        sentiment_value = find_sentiment_gpt(tweet)
        all_sentiments.append(sentiment_value)
        i = i + 1
        print(i, sentiment_value)

    except Except as e:
        print("===================")
        print("Exception occured", e)
        exception = exception + 1

print("Total exception count:", exceptions)


1 positive
2 neutral
3 negative
4 neutral
5 negative
6 positive
7 neutral
8 neutral
9 positive
10 neutral
11 neutral
12 negative
13 neutral
14 neutral
15 neutral
16 neutral
17 negative
18 neutral
19 positive
20 neutral
21 negative
22 neutral
23 neutral
24 neutral
25 positive
26 neutral
27 neutral
28 neutral
29 neutral
30 neutral
31 neutral
32 neutral
33 neutral
34 neutral
35 positive
36 positive
37 positive
38 positive
39 positive
40 positive
41 positive
42 neutral
43 positive
44 negative
45 positive
46 positive
47 positive
48 positive
49 positive
50 positive
51 negative
52 positive
53 positive
54 positive
55 negative
56 positive
57 positive
58 positive
59 positive
60 positive
61 neutral
62 positive
63 positive
64 positive
65 negative
66 negative
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 negative
74 negative
75 negative
76 negative
77 negative
78 negative
79 negative
80 negative
81 negative
82 positive
83 negative
84 negative
85 neutral
86 negative
87 n

In [65]:
accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
print("Accuracy:", accuracy)

Accuracy: 0.79


## Zero shot Text Classification with Claude 3 Opus

In [10]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key = os.environ.get('CLAUDE_API_KEY')
)

In [13]:
def find_sentiment_claude(tweet):

    content = """What is the sentiment expressed in the following tweet about an airline?
    Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
    tweet: {}""".format(tweet)

    sentiment = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0.0,
        messages=[
            {"role": "user", "content": content}
        ]
    )

    return sentiment.content[0].text

In [15]:

%%time

all_sentiments = []

tweets_list = dataset["text"].tolist()

i = 0
exceptions = 0
while i < len(tweets_list):

    try:
        tweet = tweets_list[i]
        sentiment_value = find_sentiment_claude(tweet)
        all_sentiments.append(sentiment_value)
        i = i + 1
        print(i, sentiment_value)

    except Except as e:
        print("===================")
        print("Exception occured", e)
        exception = exception + 1

print("Total exception count:", exceptions)

accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
print("Accuracy:", accuracy)

1 negative
2 neutral
3 positive
4 negative
5 positive
6 neutral
7 positive
8 neutral
9 negative
10 positive
11 negative
12 neutral
13 negative
14 neutral
15 neutral
16 neutral
17 negative
18 neutral
19 neutral
20 negative
21 neutral
22 neutral
23 positive
24 negative
25 negative
26 negative
27 positive
28 negative
29 positive
30 neutral
31 positive
32 neutral
33 negative
34 positive
35 positive
36 positive
37 negative
38 negative
39 positive
40 positive
41 positive
42 positive
43 positive
44 positive
45 positive
46 positive
47 positive
48 positive
49 positive
50 positive
51 positive
52 positive
53 neutral
54 neutral
55 positive
56 neutral
57 positive
58 positive
59 positive
60 positive
61 positive
62 positive
63 positive
64 neutral
65 positive
66 positive
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 negative
74 negative
75 negative
76 negative
77 negative
78 negative
79 negative
80 negative
81 negative
82 negative
83 negative
84 negative
85 neutral
86 nega